# Embbeding Models
In this notebook we are expermenting on two embedding models. One is the default text-embedding-ada-002 and the other is text-embedding-3-large. By improving the embedding model we can improve the relevancy of the retrived documents.

In [ ]:
# Importing libraries
import sys
from dotenv import load_dotenv
import pandas as pd
import os
import shutil
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_experimental.text_splitter import SemanticChunker
from typing import List
import pandas as pd
sys.path.insert(1, '/home/jabez/week_11/Contract-Advisor-RAG')
load_dotenv()
sys.path.insert(1, '/home/jabez/rizzbuzz with poetry/RAG-Optimization-System/scripts')
import file_loader 
import pipelines 
import evaluation

In [2]:
# loading data
file_path = '/home/jabez/rizzbuzz with poetry/RAG-Optimization-System/data/cnn_dailymail_3.0.0.csv'
data = file_loader.load_csv(file_path)

### text-embedding-ada-002

In [4]:
# Create or load a Chroma database
embeddings = OpenAIEmbeddings()
db_default = Chroma(persist_directory="/home/jabez/rizzbuzz with poetry/RAG-Optimization-System/vector_store", embedding_function=embeddings)

In [ ]:
# Generate syntetic test data
file_path = '/home/jabez/rizzbuzz with poetry/RAG-Optimization-System/test_data/syntetic_test_data.csv'
syntetic_test_data =evaluation.generate_syntetic_testdata(data, file_path)

# Loading syntetic test data
syntetic_test_data = pd.read_csv('/home/jabez/rizzbuzz with poetry/RAG-Optimization-System/test_data/syntetic_test_data.csv')

# Setting retriever
retriver = db_default.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [6]:
# Adding answer to test data from simple pipeline
file_path_with_answer = '/home/jabez/rizzbuzz with poetry/RAG-Optimization-System/test_data/syntetic_test_data_with_answer.csv'
syntetic_test_data_with_answer = evaluation.adding_answer_to_testdata(syntetic_test_data, pipelines.simple_pipeline, db_default, retriver)

# Evaluating the test data from simple pipeline
simple_rag_evaluation_result = evaluation.ragas_evaluator(syntetic_test_data_with_answer)

# Evaluation mean
result = evaluation.evaluation_mean(simple_rag_evaluation_result)

/home/jabez/rizzbuzz with poetry/RAG-Optimization-System/.venv/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
Evaluating: 100%|██████████| 80/80 [00:34<00:00,  2.33it/s]


context_precision: 89.93%, faithfulness: 84.82%, answer_relevancy: 82.21%, context_recall: 92.33%


### text-embedding-3-large

In [ ]:
# Initialize the text splitter with the specified chunk size and overlap
chunk_size= 500
chunk_overlap= 150
persist_directory = '/home/jabez/rizzbuzz with poetry/RAG-Optimization-System/text_embedding_large'
vectorstore_character = file_loader.character_text_splitter_large_embedding(data, chunk_size, chunk_overlap, persist_directory)

In [7]:
# Create or load a Chroma database

embeddings = OpenAIEmbeddings()
db_large = Chroma(persist_directory="/home/jabez/rizzbuzz with poetry/RAG-Optimization-System/text_embedding_large", embedding_function=embeddings)

In [8]:
# Generate syntetic test data
file_path = '/home/jabez/rizzbuzz with poetry/RAG-Optimization-System/test_data/syntetic_test_data.csv'
syntetic_test_data =evaluation.generate_syntetic_testdata(data, file_path)

# Loading syntetic test data
syntetic_test_data = pd.read_csv('/home/jabez/rizzbuzz with poetry/RAG-Optimization-System/test_data/syntetic_test_data.csv')

# Setting retriever
retriver = vectorstore_character.as_retriever(search_type="similarity", search_kwargs={"k": 6})

# Adding answer to test data from simple pipeline
file_path_with_answer = '/home/jabez/rizzbuzz with poetry/RAG-Optimization-System/test_data/syntetic_test_data_with_answer.csv'
syntetic_test_data_with_answer = evaluation.adding_answer_to_testdata(syntetic_test_data, pipelines.simple_pipeline, vectorstore_character, retriver)

# Evaluating the test data from simple pipeline
simple_rag_evaluation_result = evaluation.ragas_evaluator(syntetic_test_data_with_answer)

# Evaluation mean
result = evaluation.evaluation_mean(simple_rag_evaluation_result)

Filename and doc_id are the same for all nodes.                     
Evaluating: 100%|██████████| 80/80 [00:36<00:00,  2.18it/s]


context_precision: 88.17%, faithfulness: 75.8%, answer_relevancy: 84.84%, context_recall: 80.42%
